First of all let's put the required imports:


*   **Time**: This is just for finding out the time needed for the Algorithm.
*   **NumPy**: This will be our main module as we'll work with NumPy Arrays mostly and it will also be required for finding out Mean and Standard Deviation.
*   **AstroPy (FITS Module)**: We need the FITS Module of AstroPy for working with .fits files which are mostly used for Astronomical Images.





In [ ]:
import time
import numpy as np
from astropy.io import fits

Next we define a Running Stats function which implements the Welford's Method for finding out the running mean and standard deviation of each pixel in the FITS Image. The fits module is used here to open a fits file.

In [ ]:
def running_stats(filenames):
  
  n = 0
  for filename in filenames:
    hdulist = fits.open(filename)
    data = hdulist[0].data
    if n == 0:
      mean = np.zeros_like(data)
      s = np.zeros_like(data)

    n += 1
    delta = data - mean
    mean += delta/n
    s += delta*(data - mean)
    hdulist.close()

  s /= n - 1
  np.sqrt(s, s)

  if n < 2:
    return mean, None
  else:
    return mean, s

Now, we will define a function median_bins_fits which finds the number of values in each bin for a pixel. The left_bin includes the values that are less then mean - standard deviation. After this there are B bins that each include a range of values. Our median will be more accurate if B is increased but the run time of this Algorithm would also increase.

In [ ]:
def median_bins_fits(filenames, B):
  # Calculate the mean and standard dev
  mean, std = running_stats(filenames)
    
  dim = mean.shape # Dimension of the FITS file arrays
    
  # Initialise bins
  left_bin = np.zeros(dim)
  bins = np.zeros((dim[0], dim[1], B))
  bin_width = 2 * std / B 

  # Loop over all FITS files
  for filename in filenames:
      hdulist = fits.open(filename)
      data = hdulist[0].data

      # Loop over every point in the 2D array
      for i in range(dim[0]):
        for j in range(dim[1]):
          value = data[i, j]
          mean_ = mean[i, j]
          std_ = std[i, j]

          if value < mean_ - std_:
            left_bin[i, j] += 1
                
          elif value >= mean_ - std_ and value < mean_ + std_:
            bin = int((value - (mean_ - std_))/bin_width[i, j])
            bins[i, j, bin] += 1

  return mean, std, left_bin, bins

Finally we define the median_approx_fits function which calculates the actual median.

In [ ]:
def median_approx_fits(filenames, B):
  mean, std, left_bin, bins = median_bins_fits(filenames, B)
    
  dim = mean.shape # Dimension of the FITS file arrays
    
  # Position of the middle element over all files
  N = len(filenames)
  mid = (N + 1)/2
	
  bin_width = 2*std / B
  # Calculate the approximated median for each array element
  median = np.zeros(dim)   
  for i in range(dim[0]):
    for j in range(dim[1]):    
      count = left_bin[i, j]
      for b, bincount in enumerate(bins[i, j]):
        count += bincount
        if count >= mid:
          # Stop when the cumulative count exceeds the midpoint
          break
      median[i, j] = mean[i, j] - std[i, j] + bin_width[i, j]*(b + 0.5)
      
  return median